In [3]:
import pandas as pd

df = pd.read_csv('dataFrame.csv') ##check query.sql for the data source

cex = pd.read_csv('cex-addresses.csv') ##forked from https://github.com/ohhkaneda/cex-addresses-ethereum/blob/main/src/all-addresses.json

In [ ]:
#method 1: based on hourly truncated deposit times
#Truncates timestamp to hour
df['FIRST_DEPOSIT_DATE_hour'] = df.FIRST_DEPOSIT_DATE.astype('<M8[h]')

#  Counts all addresses that made their first sturdy deposit in the same hour (i.e. between 6PM and 7PM on 2022/10/01)
## And were funded by the same address
suspects = df.groupby('FIRST_DEPOSIT_DATE_hour')['FROM_ADDRESS'].value_counts().reset_index(allow_duplicates = True)

##rename columns for merging with main df
suspects.columns = ['FIRST_DEPOSIT_DATE_hour','FROM_ADDRESS','FROM_ADDRESS_count']

#Merge addresses funded by suspects that made their first deposits on the suspect deposit hours
df = df.merge(suspects, how='inner', on=['FIRST_DEPOSIT_DATE_hour','FROM_ADDRESS'])

#Merge addresses funded by CEX
df = df.merge(cex, how='left', left_on = 'FROM_ADDRESS', right_on='address')
df['name-tag'] = df['name-tag'].str.replace(' ','').str.replace('[0-9]','', regex = True) 

suspectsCex = df.groupby('FIRST_DEPOSIT_DATE_hour')['name-tag'].value_counts().reset_index(allow_duplicates = True)
suspectsCex.columns = ['FIRST_DEPOSIT_DATE_hour','name-tag','nameTagCount']

df = df.merge(suspectsCex, how='left', on=['FIRST_DEPOSIT_DATE_hour','name-tag'])

##output
df[(df['FROM_ADDRESS_count']>1) | (df['nameTagCount'] > 1)].to_csv('suspectsWithCex.csv')

In [4]:
#method 2: based on rolling deposit times, to avoid hourly cutoff
from datetime import datetime, timedelta

suspicious_addresses = []

for i,r in df.iterrows():
    
    #determine the [-1hr,+1hr] bounds 
    lowerBound = datetime.strptime(r['FIRST_DEPOSIT_DATE'][:19], '%Y-%m-%d %H:%M:%S') - timedelta(hours=1, minutes=0)
    upperBound = datetime.strptime(r['FIRST_DEPOSIT_DATE'][:19], '%Y-%m-%d %H:%M:%S') + timedelta(hours=1, minutes=0)
    
    #count distinct funding addresses for deposits around the current deposit
    suspects = df[
        (df.FIRST_DEPOSIT_DATE.astype('<M8[s]') > lowerBound) & (df.FIRST_DEPOSIT_DATE.astype('<M8[s]') < upperBound)
    ]['FROM_ADDRESS'].value_counts().reset_index(allow_duplicates = True)
    
    suspects.columns = ['address','fromAddressCount']
    
    #add suspects to the list
    suspicious_addresses.extend( 
        df[
            (df.FIRST_DEPOSIT_DATE.astype('<M8[s]') > lowerBound) 
            & (df.FIRST_DEPOSIT_DATE.astype('<M8[s]') < upperBound)
        ].merge(
            suspects[suspects.fromAddressCount > 1], how = 'inner', left_on = 'FROM_ADDRESS', right_on = 'address'
        )['USER'].to_list()
    )
    
##Remove duplicates
suspicious_addresses = list(set(suspicious_addresses))

#output suspects
suspicious_addresses

['0x1501a306356a9ca71f66985dbff6d1978e2c2c79',
 '0x33482803274c5e08661843636dcee8e9a68c7252',
 '0x89c505b005d1ca6d1f5d8533f906e0f0d513a61a',
 '0x7a4c8e3655d665f3e25ccd6b60b1b024629b3705',
 '0x2875403e0d69641cf631f10e48e63c40e4b27fc3',
 '0x6c14cdc6b4b935614d2aa42635ed79b4617da0f5',
 '0x8b1af9cf98dc5e8ac3c24b4dfefe019b2d3f24cb',
 '0x4df599c19ec1c904994115b2c23329a3ef2fb928',
 '0x98a97c80447bd8a5a84d487651728a98d26e85ef',
 '0xac3c3c2c533619568bc34a331d3fb87121bf79ab',
 '0x0388eaf12db6b8a977525d0ee0ec8f964f8aa9db',
 '0x5840d518098a0790c18a5b4ef2aaf20275cab1a4',
 '0x1a62e6f3fd7574666b84efd59763e807988829f5',
 '0x1429bb62c3fc8cbfcbddf88e49e04bd93be00670',
 '0x15bd6e69f1f895ecc2f253156f76c6bd069f5e3f',
 '0x97339369814b53952eab6f3a8a37733a81df3257',
 '0x1d88f5a58624688627fccd370dc8eac3a700c138',
 '0xc9a5a7fda5f10f701e74902cade11b104fe30fbe',
 '0xffebf6747d8b46852e552d0ffebe71d47ac9fcae',
 '0x66787a9aa595a75168018bc9137c9907dd54483b',
 '0xcff1e58f8286a69ee09c25aa47f153f2d4c6f588',
 '0x616b14129